In [3]:
import os
import string
import numpy as np
import pandas as pd
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import logging
import tensorflow as tf

import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import io
import time
import warnings
import sys

## Data Sets

In [ ]:
!unzip '/project/code/deepak/Pytorch_Tutorial/Machine_T/NEWS2018_DATASET_04.zip'

In [ ]:
# Importing the required libraries
import xml.etree.ElementTree as Xet
import pandas as pd

cols = ["SourceName", "TargetName"]
rows = []

# Parsing the XML file
xmlparse = Xet.parse('/project/datasets/code/deepak/Pytorch_Tutorial/Machine_T/NEWS2018_DATASET_04/NEWS2018_M-EnHi_train.xml')
root = xmlparse.getroot()
for i in root:
    SourceName = i.find("SourceName").text
    TargetName = i.find("TargetName").text


    rows.append({"SourceName": SourceName,
                 "TargetName": TargetName,
                })

df = pd.DataFrame(rows, columns=cols)

# Writing dataframe to csv
df.to_csv('Output.csv')


In [ ]:
# Importing the required libraries
import xml.etree.ElementTree as Xet
import pandas as pd

cols = ["SourceName", "TargetName"]
rows = []

# Parsing the XML file
xmlparse = Xet.parse('/project/datasets/code/deepak/Pytorch_Tutorial/Machine_T/NEWS2018_DATASET_04/NEWS2018_M-EnHi_dev.xml')
root = xmlparse.getroot()
for i in root:
    SourceName = i.find("SourceName").text
    TargetName = i.find("TargetName").text


    rows.append({"SourceName": SourceName,
                 "TargetName": TargetName,
                })

df = pd.DataFrame(rows, columns=cols)

# Writing dataframe to csv
df.to_csv('Validation_data.csv')


## Text Preprocessing

In [4]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    return w

def hindi_preprocess_sentence(w):
    w = w.rstrip().strip()
    return w

In [5]:
def create_dataset(path):
    lines=pd.read_csv(path,index_col =[0])
    lines=lines.dropna()
    
    en = []
    hd = []
    for i, j in zip(lines['SourceName'], lines['TargetName']):
        en_1 = [preprocess_sentence(w) for w in i.split(' ')]
        en_1.append('<end>')
        en_1.insert(0, '<start>')
        hd_1 = [hindi_preprocess_sentence(w) for w in j.split(' ')]
        hd_1.append('<end>')
        hd_1.insert(0, '<start>')
        en.append(en_1)
        hd.append(hd_1)
    return hd, en

In [6]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [7]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
  return tensor, lang_tokenizer

In [41]:
train_data_path = '/project/datasets/code/deepak/Pytorch_Tutorial/Machine_T/output.csv'

In [31]:
val_data_path = '/project/datasets/code/deepak/Pytorch_Tutorial/Machine_T/Validation_data.csv'

In [33]:
train_data = pd.read_csv(train_data_path)

In [34]:
val_data = pd.read_csv(val_data_path)

In [35]:
whole_data = pd.concat([train_data,val_data])

In [37]:
whole_data.to_csv('whole_data.csv')

In [38]:
data_path = '/project/datasets/code/deepak/Pytorch_Tutorial/Machine_T/whole_data.csv'

In [42]:
def load_dataset(path):
    targ_lang, inp_lang = create_dataset(data_path)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [43]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(train_data_path)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [44]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

11149 11149 2788 2788


In [45]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
    
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
11631 ----> shivlal
1785 ----> yadav
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
11247 ----> शिवलाल
2058 ----> यादव
2 ----> <end>


In [46]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 256
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
## En

In [47]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [48]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    hidden_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, attention_weights

In [49]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)
    return x, state, attention_weights

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [50]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
#   print(type(mask))
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [51]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [52]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
    # Teacher forcing
    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))      
  return batch_loss

In [60]:
EPOCHS = 200

for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.5518
Epoch 1 Batch 100 Loss 0.6290
Epoch 1 Loss 0.5952
Time taken for 1 epoch 4.58247709274292 sec

Epoch 2 Batch 0 Loss 0.5508
Epoch 2 Batch 100 Loss 0.5628
Epoch 2 Loss 0.5711
Time taken for 1 epoch 4.706860780715942 sec

Epoch 3 Batch 0 Loss 0.5245
Epoch 3 Batch 100 Loss 0.5545
Epoch 3 Loss 0.5503
Time taken for 1 epoch 4.525805711746216 sec

Epoch 4 Batch 0 Loss 0.4945
Epoch 4 Batch 100 Loss 0.5094
Epoch 4 Loss 0.5254
Time taken for 1 epoch 4.831615447998047 sec

Epoch 5 Batch 0 Loss 0.4839
Epoch 5 Batch 100 Loss 0.4852
Epoch 5 Loss 0.5004
Time taken for 1 epoch 4.548644781112671 sec

Epoch 6 Batch 0 Loss 0.4165
Epoch 6 Batch 100 Loss 0.4777
Epoch 6 Loss 0.4784
Time taken for 1 epoch 4.667583703994751 sec

Epoch 7 Batch 0 Loss 0.4188
Epoch 7 Batch 100 Loss 0.4551
Epoch 7 Loss 0.4589
Time taken for 1 epoch 4.584505796432495 sec

Epoch 8 Batch 0 Loss 0.4056
Epoch 8 Batch 100 Loss 0.4243
Epoch 8 Loss 0.4412
Time taken for 1 epoch 4.650224924087524 sec

Epoch 9 B

In [61]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_sentence(sentence)
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [62]:
def translate(sentence):
    result, sentence = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [63]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [65]:
translate('zorion')

Input: zorion
Predicted translation: औरव <end> 


In [69]:
translate('aachaaryanandan')

Input: aachaaryanandan
Predicted translation: अर्पणा <end> 
